In [1]:
import pandas as pd
import altair as alt
import numpy as np
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import visibility_of_element_located

scriptures = pd.read_csv("C:/Users/LeRoy12358/Documents/School/W23/499 DS - Senior Project/WWPapers/raw data/lds-scriptures.csv")

# scriptures

In [2]:
browser = webdriver.Chrome()  # start a web browser
browser.get("https://scriptures.byu.edu/#:t3d092:p573")  # navigate to URL
# wait for page to load
# by waiting for <h1> element to appear on the page
title = (
    WebDriverWait(driver=browser, timeout=10)
    .until(visibility_of_element_located((By.ID, "jContent")))
    .text
)
# retrieve fully rendered HTML content
content = browser.page_source
browser.close()



In [3]:
# # we then could parse it with beautifulsoup
soup = BeautifulSoup(content, "html.parser")
# print(soup.find("jContent").text)
# soup

# soup.div['id']

# print(soup.prettify())
# content
# soup.div.div.div

soup.find("div", {"id": "jContent"})
tmp = soup.find('div', {'class': 'discourseBody'}).text

# tmp.replace("\xa0", "")
# tmp

In [4]:
links = soup.find('ul', {'class': 'talksblock'})

linkList = links.find_all('a', {'class': 'refcounter'})

link1 = links.find('a', {'class': 'refcounter'})['onclick']

# type(link1)
link1.replace("getTalk('", "").replace("');", "")
# for item in linkList:
#     # item.replace("getTalk('", "").replace("');", "")
#     # print(item)
#     pass
# linkList.replace("getTalk('", "").replace("');", "")

linkList2 = []

for link in soup.find_all('a', {'class': 'refcounter'}):
    linkList2.append(
        np.base_repr(int(
        link.get('onclick').replace("getTalk('", "").replace("');", "")
        ), 16
        ).lower()
        )
# http://example.com/elsie
# http://example.com/lacie
# http://example.com/tillie

linkList2

['3d092',
 '3a99c',
 '3a987',
 '38296',
 '3827e',
 '38279',
 '35b8e',
 '35b8d',
 '35b8b',
 '35b75',
 '35b7e',
 '35b72',
 '35b79',
 '33473',
 '33471',
 '3346f',
 '33466',
 '3345f',
 '2e662',
 '2e65a',
 '2e648',
 '2e653',
 '2bf3b',
 '2bf36',
 '2bf2f',
 '2bf2e',
 '2bf25',
 '29834',
 '2982e',
 '2981b',
 '27125',
 '27106',
 '24a1a',
 '24a13',
 '249fc',
 '249f2',
 '222e1',
 '222e6',
 '1fbe3',
 '1fbf5',
 '1d4f7',
 '1d4c4',
 '1ade7',
 '1add5',
 '1adba',
 '186cc',
 '186a3',
 '15fd2',
 '15faf',
 '15f9c',
 '138c6',
 '138c5',
 '1117f',
 'ea75',
 'ea72',
 'c37b',
 'c363',
 '9c7a',
 'c35b',
 '9c6c',
 '9c66',
 '9c5f',
 '9c54',
 '15fb9',
 '4e41']

The try block lets you test a block of code for errors.

The except block lets you handle the error.

The else block lets you execute code when there is no error.

The finally block lets you execute code, regardless of the result of the try- and except blocks.

In [5]:
# This will likely take quite a while to run....

# breaker = False
browser = webdriver.Chrome()  # start a web browser
pot = []
rawPot = []

for i in range(0, len(linkList2)):
    url = linkList2[i]
    print(url, end = ", ")
    
    try:


        browser.get("https://scriptures.byu.edu/#:t" + url + ":p573")  # navigate to URL
        # wait for page to load by waiting for <h1> element to appear on the page
        title = (
            WebDriverWait(driver=browser, timeout=10)
            .until(visibility_of_element_located((By.ID, "jContent")))
            .text
        )
        # retrieve fully rendered HTML content
        content = browser.page_source

    except:
        print("Something broke, autofixing....", end = " ")
        browser.close()
        browser = webdriver.Chrome()
        i -= 1
        # continue
    else:
        # we then could parse it with beautifulsoup
        soup = BeautifulSoup(content, "html.parser")
        rawPot.append(soup.find('div', {'class': 'discourseBody'}))
        pot.append(soup.find('div', {'class': 'discourseBody'}).text)
        # pot.append(soup.find_all('a', {'href': 'discourseBody'}))
        time.sleep(.11)
    
browser.close()
    # if(breaker):
    #     break
    # breaker = True
    

3d092, 3a99c, 3a987, 38296, Something broke, autofixing.... 3827e, 38279, 35b8e, Something broke, autofixing.... 35b8d, 35b8b, Something broke, autofixing.... 35b75, 35b7e, 35b72, 35b79, 33473, 33471, 3346f, 33466, 3345f, Something broke, autofixing.... 2e662, 2e65a, 2e648, 2e653, 2bf3b, 2bf36, 2bf2f, 2bf2e, 2bf25, 29834, 2982e, 2981b, Something broke, autofixing.... 27125, 27106, 24a1a, 24a13, 249fc, Something broke, autofixing.... 249f2, 222e1, 222e6, 1fbe3, 1fbf5, 1d4f7, 1d4c4, Something broke, autofixing.... 1ade7, 1add5, 1adba, 186cc, 186a3, 15fd2, Something broke, autofixing.... 15faf, 15f9c, 138c6, 138c5, Something broke, autofixing.... 1117f, ea75, ea72, c37b, c363, 9c7a, Something broke, autofixing.... c35b, 9c6c, 9c66, Something broke, autofixing.... 9c5f, 9c54, 15fb9, 4e41, 

In [19]:
# print(rawPot[0])

In [29]:
# tmp = pot[0].find('div', {'class': 'discourseBody'}).text
tmp = rawPot[0]

# soup.find("div", {"id": "jContent"})
# tmp = soup.find('div', {'class': 'discourseBody'}).text

# tmp.find_all('a', {'href': 'javascript:void(0)'})#'onclick': 'gs(\d*)'})
tmp.find_all('span', {'class': 'citation'})#'onclick': 'gs(\d*)'})

citationList = tmp.find_all('span', {'class': 'citation'})



referenceList = []

for citation in citationList:
    referenceList.append(
    citation.find_all('a', {'href': 'javascript:void(0)'})[1]
    )
    # print(citation.find('span', {'class': 'citation'}))

# # TODO: this loop doesn't actually form a referenceList of all references in the html file. Instead, it takes the first reference from each file. The above loop should be correct
# for rawSoup in rawPot:
#     referenceList.append(
#     rawSoup.find('span', {'class': 'citation'}).find_all('a', {'href': 'javascript:void(0)'})[1].text
#     )
#     print(rawSoup.find('span', {'class': 'citation'}))

# # tmp.find('span', {'class': 'citation'}).find_all('a', {'href': 'javascript:void(0)'})[1].text

print(referenceList)

[<a href="javascript:void(0)" onclick="gs(81044)">1 Ne. 13:12</a>, <a href="javascript:void(0)" onclick="gs(92013)">D&amp;C 135:3</a>, <a href="javascript:void(0)" onclick="gs(92090)">D&amp;C 136:39</a>, <a href="javascript:void(0)" onclick="gs(77079)">Heb. 11:37</a>, <a href="javascript:void(0)" onclick="gs(79274)">Rev. 6:9</a>, <a href="javascript:void(0)" onclick="gs(78347)">2 Pet. 1:21</a>, <a href="javascript:void(0)" onclick="gs(52425)">Gen. 2:7</a>, <a href="javascript:void(0)" onclick="gs(58970)">Isa. 31:1</a>, <a href="javascript:void(0)" onclick="gs(57425)">Prov. 14:34</a>, <a href="javascript:void(0)" onclick="gs(74706)">Gal. 6:7</a>, <a href="javascript:void(0)" onclick="gs(63721)">Matt. 7:2</a>, <a href="javascript:void(0)" onclick="gs(128980)">Isa. 40:1-3</a>, <a href="javascript:void(0)" onclick="gs(128981)">Jer. 31:1-14</a>, <a href="javascript:void(0)" onclick="gs(128982)">Ezek. 20:33-44</a>, <a href="javascript:void(0)" onclick="gs(83841)">D&amp;C 5:14</a>, <a href="j

In [7]:
pd.set_option('display.max_rows', 100)

s = pd.Series(pot)
# s = pd.Series([tmp.replace("\xa0", "")], index=[""])
s = s.str.replace("(,|\.|;) *", "\\1 ")

pd.DataFrame(s.str.extractall(
    # "(.{0,30})(\d* *\w*&?\w*\.? *\d+:\d+)"
    "(.{0,30})(\d+:\d+)"
    )).reset_index().drop(columns=[])

s[0]

C:\Users\LeRoy12358\AppData\Local\Temp\ipykernel_13308\3275803033.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace("(,|\.|;) *", "\\1 ")


"\n There is a proverb or saying which I have heard a good many times in my life, and which I think bears a great deal of weight, and that is, “truth is mighty and will prevail. ” I think this has been manifested in every capacity in which truth has been used, whether applied temporally or spiritually; whether applied in the capacity of nations, or families, or individuals; whether applied to the world or to the kingdom of God. I think that in every age of the world, truth, whether it has been popular or unpopular, has proved itself, in the end of its labor, to prevail in all cases. When Columbus was moved upon by the Spirit of God, to cross the ocean to find a new continent, \xa01 Ne. 13:12 his object and desires were unpopular with those by whom he was surrounded, and it was only after a good deal of labor that he gained favor in the eyes of any of the rulers of the nations [p. \xa04b] whereby to receive assistance sufficient to carry out his wishes. But in the end he prevailed. He f

In [16]:
pd.DataFrame(s.str.extractall(
    "(.{0,30})(\d* *\w*&?\w*\.? *\d+:\d+-?\d*)"
    )).reset_index().drop(columns=[])


,level_0,match,0,1
0,0,0,"ean to find a new continent,",1 Ne. 13:12
1,0,1,testimony with their blood.,D&C 135:3
2,0,2,D&C 13,6:39
3,0,3,"d; others were sawn asunder,",Heb. 11:37
4,0,4,e testimony of Jesus Christ.,Rev. 6:9
...,...,...,...,...
2630,53,64,showing that summer is nigh.,Matt. 24:32
2631,53,65,"e among the foolish virgins,",Matt. 25:1
2632,53,66,-13 D&C 6,3:54
2633,53,67,"tand the signs of the times,",Matt. 16:3


In [ ]:
from difflib import SequenceMatcher

def findMatches(str1, str2, BLOCK_MIN_LENGTH = 6):
    sm = SequenceMatcher(lambda x: x in " ", str1, str2)
    finalMatches = []
    matchesPosns = sm.get_matching_blocks()

    # put together a list of matches longer than only a couple characters
    for matchPos in matchesPosns:
        matchLength = matchPos[2]
        if matchLength < BLOCK_MIN_LENGTH:
            continue
        # print(str1[matchPos[0]:matchPos[0] + matchPos[2]])
        finalMatches.append(matchPos)
    return [str1[finalMatches[0][0]:finalMatches[-1][0] + finalMatches[-1][2]],
            str2[finalMatches[0][1]:finalMatches[-1][1] + finalMatches[-1][2]]]


In [ ]:
myStr1 = "He has left on record through the medium of holy men who wrote and spoke as they were moved upon by the Holy Ghost"
myStr2 = "For the prophecy came not in old time by the will of man: but holy men of God spake as they were moved by the Holy Ghost."
findMatches(myStr1, myStr2)




[' holy men who wrote and spoke as they were moved upon by the Holy Ghost',
 ' holy men of God spake as they were moved by the Holy Ghost']